# Settings & Options

PowerWorld has many simulation settings — solver iteration limits,
convergence tolerances, control flags, GIC parameters, and more.
Traditionally these require verbose getter/setter calls through
the SimAuto API. ESA++ exposes the most commonly used settings as
**Python attributes** on `PowerWorld` and `pw.gic`, so reading or
changing a setting is as simple as reading or assigning a variable.

```python
from esapp import PowerWorld
from esapp.components import *

pw = PowerWorld("path/to/case.pwb")
```

In [1]:
from esapp import PowerWorld
from esapp.components import *
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

'open' took: 7.1384 sec


## Solver Options

Solver options control how the Newton-Raphson power flow behaves —
iteration limits, convergence criteria, control checking, and
special modes like DC approximation. They live directly on the
`PowerWorld` object.

**Boolean options** return `True`/`False` and accept `True`/`False`.
**Numeric options** return and accept `int` or `float` values.

### Reading options

Just access the attribute — no method call needed.

In [2]:
# Read current solver settings
pw.max_iterations, pw.convergence_tol, pw.flat_start, pw.dc_mode

(50, 1e-07, False, False)

### Changing options

Assign a new value and it takes effect immediately in PowerWorld.
Changes persist for the life of the session (or until you change
them again).

In [3]:
# Tighten convergence and increase iteration limit
pw.max_iterations = 200
pw.convergence_tol = 1e-5

# Verify the changes took effect
assert pw.max_iterations == 200
assert pw.convergence_tol == 1e-5

pw.max_iterations, pw.convergence_tol

(200, 1e-05)

In [4]:
# Toggle boolean options
pw.do_one_iteration = True
assert pw.do_one_iteration is True

pw.do_one_iteration = False
assert pw.do_one_iteration is False

# Enable DC power flow mode, then disable
pw.dc_mode = True
assert pw.dc_mode is True
pw.dc_mode = False

### Available solver options

Here is the full list of solver options exposed on `PowerWorld`:

**Iteration & convergence:**

| Attribute | Type | Description |
|---|---|---|
| `max_iterations` | int | Maximum Newton-Raphson iterations |
| `max_vcl_iterations` | int | Maximum voltage-constrained loop iterations |
| `convergence_tol` | float | Power flow convergence tolerance |
| `min_volt_i_load` | float | Minimum voltage for constant-current loads (pu) |
| `min_volt_s_load` | float | Minimum voltage for constant-impedance loads (pu) |

**Boolean controls:**

| Attribute | Description |
|---|---|
| `do_one_iteration` | Solve only one Newton iteration per call |
| `disable_opt_mult` | Disable optimal multiplier acceleration |
| `flat_start` | Start from flat voltage profile (1.0 pu, 0 deg) |
| `dc_mode` | Enable DC power flow approximation |
| `inner_ss_check` | Check switched shunt controls in inner loop |
| `disable_gen_mvr_check` | Disable generator MVR limit checking |
| `inner_check_gen_vars` | Check generator VAR limits in inner loop |
| `inner_backoff_gen_vars` | Back off generator VAR limits in inner loop |
| `check_taps` | Check transformer tap adjustments |
| `check_shunts` | Check switched shunt adjustments |
| `check_phase_shifters` | Check phase shifter adjustments |
| `prevent_oscillations` | Prevent control oscillations |
| `disable_angle_rotation` | Disable automatic angle rotation to slack |
| `allow_mult_islands` | Allow multiple island solutions |
| `eval_solution_island` | Evaluate solution quality per island |
| `enforce_gen_mw_limits` | Enforce generator MW output limits |

## GIC Options

GIC (Geomagnetically Induced Currents) analysis has its own set
of options, accessed through `pw.gic`. These control whether GIC
effects are included in simulations, the calculation mode, electric
field parameters, and more. They work exactly like solver options —
read by accessing the attribute, write by assigning to it.

In [5]:
# Read current GIC settings
pw.gic.pf_include, pw.gic.ts_include, pw.gic.calc_mode

(False, False, 'SnapShot')

In [6]:
# Enable GIC in power flow and set calculation mode
pw.gic.pf_include = True
pw.gic.calc_mode = 'SnapShot'

assert pw.gic.pf_include is True
assert pw.gic.calc_mode == 'SnapShot'

# Set electric field parameters
pw.gic.efield_angle = 90.0
pw.gic.efield_mag = 1.0

pw.gic.efield_angle, pw.gic.efield_mag

('90', '1')

### Quick configuration

For common GIC setups, `configure()` sets multiple options at
once with sensible defaults. Calling it with no arguments enables
GIC in power flow with snapshot mode — the most common starting
point.

In [7]:
# Apply defaults: pf_include=True, ts_include=False, calc_mode='SnapShot'
pw.gic.configure()

pw.gic.pf_include, pw.gic.ts_include, pw.gic.calc_mode

(True, False, 'SnapShot')

In [8]:
# Or customize: enable transient stability and time-varying mode
pw.gic.configure(ts_include=True, calc_mode='TimeVarying')

assert pw.gic.ts_include is True
assert pw.gic.calc_mode == 'TimeVarying'

# Reset to defaults
pw.gic.configure()

### Viewing all GIC settings

The `settings()` method returns every GIC option as a DataFrame,
including options that don't have a dedicated descriptor attribute.
This is useful for inspecting the full configuration or for
debugging.

In [9]:
pw.gic.settings().head(10)

,VariableName,ValueField
0,AutoXFMaxTurnsRatio,4
1,BusNoSub,None (Ungrounded)
2,CalcInducedDCVoltEquiv,NO
3,CalcInducedDCVoltLength,1
4,CalcInducedDCVoltLowR,NO
5,CalcMaxDirection,YES
6,CalcMode,SnapShot
7,DistXFConfigDefault,GWye
8,EField3dFileMerge,YES
9,EField3dFileMultLoad,0


### Available GIC options

**Core:**

| Attribute | Type | Description |
|---|---|---|
| `pf_include` | bool | Include GIC effects in power flow |
| `ts_include` | bool | Include GIC effects in transient stability |
| `calc_mode` | str | Calculation mode (SnapShot, TimeVarying, etc.) |

**Electric field:**

| Attribute | Type | Description |
|---|---|---|
| `efield_angle` | float | Storm direction in degrees |
| `efield_mag` | float | Field magnitude in V/distance |
| `calc_max_direction` | bool | Auto-calculate maximum E-field direction |

**Calculation controls:**

| Attribute | Type | Description |
|---|---|---|
| `update_line_volts` | bool | Auto-update line DC voltages |
| `skip_equiv_lines` | bool | Skip DC voltage on equivalent lines |
| `skip_low_r_lines` | bool | Skip DC voltage on low-R lines |
| `min_kv` | float | Minimum nominal kV to include GIC effects |
| `segment_length_km` | float | Max line segment length (km) |
| `bus_no_sub` | str | Auto-insert option for buses without substations |
| `hotspot_include` | bool | Include hotspot calculations |

## How It Works

Under the hood, these attributes are Python
[descriptors](https://docs.python.org/3/howto/descriptor.html) —
`SolverOption` and `GICOption` classes defined in
`esapp._descriptors`. Each descriptor maps a Python attribute
name to a PowerWorld option field:

- **Reading** (`pw.max_iterations`) queries PowerWorld for the
  current value of the corresponding field and returns it as a
  Python `bool`, `int`, or `float`.
- **Writing** (`pw.max_iterations = 200`) sends the new value to
  PowerWorld immediately.
- **Boolean options** are stored in PowerWorld as `"YES"`/`"NO"`
  strings but are automatically converted to/from Python
  `True`/`False`.

You can access the underlying descriptor to see which PowerWorld
field it maps to:

In [10]:
# Access the descriptor at the class level to see the PowerWorld field it maps to
desc = type(pw).max_iterations
desc.key, desc.is_bool

('MaxItr', False)